In [1]:
#Oi shoppee, i made this with just a guess but yeah anyway it might help you a bit some dummy data is included
#It's not finished i just thought i get the basics down if you wanted, it doesnt check if thyve shipped you a wrong part
#Also it assume you internally use the one code or whatever, but it does the comparing via a master sheet like we spoke about
#Cheers

# Order Delivery Comparison Demo
# By Connor Mclean
# Made to check whether order form is same as delivery
# If discrepencies exist, will create a new file listing them

# NOTE: that this assumes data is in the same format
# If data is not in the same format, some tinkering would be required

In [2]:
# Importing Various libraries

# Using Pandas dataframe structure for processing
import pandas as pd
pd.options.mode.chained_assignment = None

# Using system to halt incase of errors
import sys

#Decimal for money conversion
from decimal import Decimal

# Importing Data 
# NOTE: all data is in CSV form at the moment

OrderSheet = pd.read_csv('DummyData\DummyOrderDataCSV-V2.csv', encoding = 'ISO-8859-1')
MasterSheet = pd.read_csv('DummyData\DummyMasterSheetCSV-V2.csv', encoding = 'ISO-8859-1')
DeliverySheet = pd.read_csv('DummyData\DummyDeliveryDataCSV-V2.csv', encoding = 'ISO-8859-1')


In [3]:

#Dropping empty Columns and Rows
MasterSheet = MasterSheet.dropna(axis=0, how='all')
MasterSheet = MasterSheet.dropna(axis=1, how='all')
DeliverySheet = DeliverySheet.dropna(axis=0, how='all')
OrderSheet = OrderSheet.dropna(axis = 0, how="all")

#Convert Dollars to float for comparison, both per item and total
OrderSheet['Buy Cost (Item)'] = OrderSheet["Buy Cost (Item)"].str.replace('[$]', '')
OrderSheet["Buy Cost (Item)"] = OrderSheet["Buy Cost (Item)"].str.replace(',', '')
OrderSheet["Buy Cost (Item)"] = pd.to_numeric(OrderSheet["Buy Cost (Item)"], errors='raise')

OrderSheet["Buy Cost (Total)"] = OrderSheet["Buy Cost (Total)"].str.replace('[$]', '')
OrderSheet["Buy Cost (Total)"] = OrderSheet["Buy Cost (Total)"].str.replace(',', '')
OrderSheet["Buy Cost (Total)"] = pd.to_numeric(OrderSheet["Buy Cost (Total)"], errors='raise')

DeliverySheet["Buy Cost (Item)"] = DeliverySheet["Buy Cost (Item)"].str.replace('[$]', '')
DeliverySheet["Buy Cost (Item)"] = DeliverySheet["Buy Cost (Item)"].str.replace(',', '')
DeliverySheet["Buy Cost (Item)"] = pd.to_numeric(DeliverySheet["Buy Cost (Item)"], errors='raise')

DeliverySheet["Buy Cost (Total)"] = DeliverySheet["Buy Cost (Total)"].str.replace('[$]', '')
DeliverySheet["Buy Cost (Total)"] = DeliverySheet["Buy Cost (Total)"].str.replace(',', '')
DeliverySheet["Buy Cost (Total)"] = pd.to_numeric(DeliverySheet["Buy Cost (Total)"], errors='raise')


In [5]:
OPartCode = 0
MPartCodeRow = 0
MPartCodeCol = 0
DPartCode = 0

f = open('Discrepencies.txt', 'w')

#Loop through Order sheet
while OPartCode < len(OrderSheet):
    MPartCodeRow = 0
    #Loop through Master SHeet
    while MPartCodeRow < len(MasterSheet):

        MPartCodeCol = 0
        #If Codes match with mastersheet
        if OrderSheet['Part#'][OPartCode] == MasterSheet['Our code:'][MPartCodeRow]:
            
            #Loop through row that contains code and compare to DeliverySheet
            while MPartCodeCol < MasterSheet.shape[1]:
                
                DPartCode = 0
                #Loop through delivery sheet looking fro possible match of master code and delivery code
                while DPartCode < len(DeliverySheet):
                    
                    if MasterSheet[MasterSheet.columns.values[MPartCodeCol]][MPartCodeRow] == DeliverySheet['Part#'][DPartCode]:
                        
                        if (DeliverySheet['Buy Cost (Total)'][DPartCode] != OrderSheet['Buy Cost (Total)'][OPartCode]) or (DeliverySheet['Supplied'][DPartCode] != OrderSheet['Ordered'][OPartCode]) or (DeliverySheet['Buy Cost (Item)'][DPartCode] != OrderSheet['Buy Cost (Item)'][OPartCode]):
                            f.write("Part: %s (Deliver Code: %s) Has the following differences: \n" %(OrderSheet['Part#'][OPartCode], DeliverySheet['Part#'][DPartCode]))
                            
                            #COMPARE PRICES AND QUANTITY TO ORDER FORM
                            if DeliverySheet['Buy Cost (Total)'][DPartCode] != OrderSheet['Buy Cost (Total)'][OPartCode]:
                                f.write("Different Buy Cost (Total). \n Expected: $%.2f \n Recieved: $%.2f \n" % (OrderSheet['Buy Cost (Total)'][OPartCode], DeliverySheet['Buy Cost (Total)'][DPartCode]))
                            
                            if DeliverySheet['Supplied'][DPartCode] != OrderSheet['Ordered'][OPartCode]:
                                f.write("Different Supplied amount (Total). \n Expected: %d \n Recieved: %d \n" % (OrderSheet['Ordered'][OPartCode], DeliverySheet['Supplied'][DPartCode]))
                      
                            if DeliverySheet['Buy Cost (Item)'][DPartCode] != OrderSheet['Buy Cost (Item)'][OPartCode]:
                                f.write("Different Buy Cost (Total). \n Expected: $%.2f \n Recieved: $%.2f \n" % (OrderSheet['Buy Cost (Item)'][OPartCode], DeliverySheet['Buy Cost (Item)'][DPartCode]))
                                
                            f.write("\n\n")
                        MPartCodeCol = MasterSheet.shape[1]
                        MPartCodeRow = len(MasterSheet)
                        break
                        
                    
                    DPartCode += 1
                    
                MPartCodeCol += 1
        
        MPartCodeRow += 1
    
    OPartCode += 1
        
f.close()